In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
import warnings
from scipy import stats
from IPython.core.pylabtools import figsize
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import (roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, 
                             precision_recall_curve)

In [ ]:
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

In [ ]:
def preprocessing_data(fileName):
    df = pd.read_csv(fileName)
    genre_list = pd.factorize(df['genre'])[1].tolist()
    genre_num = len(genre_list)
    for genre in genre_list:
        df[f'genre_{genre}'] = np.zeros(df.shape[0])
        df.loc[df['genre'] == genre, f'genre_{genre}'] = 1
    df.head()

    return df

df = preprocessing_data("final_data.csv")
df, df2 = train_test_split(df, test_size=0.5, random_state=42)
genre_list = pd.factorize(df['genre'])[1].tolist()
genre_num = len(genre_list)
acoustic_feature_list = ['dance', 'energy', 'speechiness', 'valence', 'tempo']
p_num = len(acoustic_feature_list)


In [ ]:
df.describe()

In [ ]:
acoustic_feature_list

In [ ]:
genre_list

# Bayesian Logistic Regression + Linear Regression

In [ ]:
# Logistic Regression Part

## hyperprior
mu_theta = 0
sigma_theta = 10
mu_beta = 1000
sigma_beta = 300
sigma_sigma = 1e5

theta_genre_dict = dict()
acoustic_feature_data = dict()
pz_logist_dict = dict()
logist_genre_dict = dict()

genre_idxs, genres = pd.factorize(df['genre'])

coords = {"genre" : genre_list, 
          "acoustic_feature" : ["intercept"]+acoustic_feature_list, 
          "obs_id": np.arange(df.shape[0])
}
with pm.Model(coords=coords) as main_model:

    genre_idx = pm.Data("genre_idx", genre_idxs, dims="obs_id")

    for i in range(p_num):
            acoustic_feature_data[i] = pm.Data(acoustic_feature_list[i], df[acoustic_feature_list[i]].values, dims="obs_id")

    #priors on theta
    theta_genre = pm.Normal('theta_genre', mu=mu_theta, sigma=sigma_theta, dims=("genre", "acoustic_feature"))
    for i in range(len(genre_list)):
        theta_genre_dict[i] = pm.Deterministic(f'theta_{genre_list[i]}', theta_genre[i,:])
    

    # find probability being each genre from logistic regression
    #fit the data 
    for g_idx, genre in enumerate(genre_list):
        logist_temp = theta_genre[g_idx, 0]
        for i in range(p_num):
            logist_temp = logist_temp + theta_genre[g_idx, i+1]*acoustic_feature_data[i]
        
        pz_logist_dict[genre] = pm.Deterministic(f'pz_genre_{genre}', logist_temp, dims="obs_id")
        logist_genre_dict[genre] = pm.Bernoulli(f'logist_genre_{genre}', p=pm.math.sigmoid(logist_temp), observed=df[f'genre_{genre}'], dims="obs_id")

with main_model:
    trace=pm.sample(5000, tune=2000, return_inferencedata=True, cores=4)

with main_model:
    genre_pred = pm.sample_posterior_predictive(
        trace.posterior
    )
    az.from_pymc3_predictions(
        genre_pred, idata_orig=trace, inplace=True
    )
with main_model:
    ppc = pm.sample_posterior_predictive(
        trace, var_names=[f'pz_genre_{genre}' for genre in genre_list] + [f'theta_{genre}' for genre in genre_list]
    )

In [ ]:
theta_list = [f'theta_{genre}' for genre in genre_list]
az.plot_trace(trace, var_names=theta_list)

In [ ]:
# Linear Regression Part

# save mean and sd of theta from previous part
mean_theta_post = dict()
sd_theta_post = dict()
for genre in genre_list:
    mean_theta_post[genre] = [np.mean(ppc[f'theta_{genre}'][:,x]) for x in range(1+p_num)]
    sd_theta_post[genre] = [np.std(ppc[f'theta_{genre}'][:,x]) for x in range(1+p_num)]
post_theta_dict = dict()
for g_idx, _ in enumerate(genre_list):
    post_theta_dict[g_idx] = dict()

pz_genre = dict()
beta_genre_dict = dict()

srng = tt.random.utils.RandomStream(seed=234)

coords = {"genre" : genre_list, 
          "acoustic_feature" : ["intercept"]+acoustic_feature_list, 
          "obs_id": np.arange(df2.shape[0])
}
with pm.Model(coords=coords) as main_model2:
    
    for i in range(p_num):
            acoustic_feature_data[i] = pm.Data(acoustic_feature_list[i], df2[acoustic_feature_list[i]].values, dims="obs_id")

    # use the parameter for setting the probability distribution of theta of each genre:
    for g_idx, genre in enumerate(genre_list):
        for i in range(1+p_num):
            post_theta_dict[g_idx][i] = pm.Deterministic(f'theta_genre_{genre}_{i}',srng.normal(mean_theta_post[genre][i], sd_theta_post[genre][i]))

    # find probability being each genre from logistic regression
    for g_idx, genre in enumerate(genre_list):
        logist_temp = post_theta_dict[g_idx][0]
        for i in range(p_num):
            logist_temp = logist_temp + post_theta_dict[g_idx][i+1]*acoustic_feature_data[i]
        pz_genre[g_idx] = pm.math.sigmoid(logist_temp)

    # select genre having the highest probablity
    pz_genre_stack = tt.stack([pz_genre[g_idx] for g_idx,_ in enumerate(genre_list)], axis=1)
    selected_genre = tt.argmax(pz_genre_stack, axis=1)
    saved_genre = pm.Deterministic('saved_genre', selected_genre)
    
    selected_genre_idx = selected_genre.eval()
    
    #priors on beta, sigma
    beta_genre = pm.Normal('beta_genre', mu=mu_beta, sigma=sigma_beta, dims=("genre", "acoustic_feature"))
    sigma = pm.HalfNormal('sigma', sigma=sigma_sigma)
    for i in range(len(genre_list)):
        beta_genre_dict[i] = pm.Deterministic(f'beta_{genre_list[i]}', beta_genre[i,:])
    
    # find number of views from linear regression
    y_est = beta_genre[selected_genre_idx, 0]
    for i in range(p_num):
        y_est = y_est + beta_genre[selected_genre_idx, i+1]*acoustic_feature_data[i]

In [ ]:
with main_model2:
    #fit the data of the number of views
    y = pm.Normal('popularity', mu=y_est, sigma=sigma, observed=df2['popularity'], dims="obs_id")
    
    trace2=pm.sample(5000, tune=2000, target_accept=0.95, return_inferencedata=True, cores=2)
with main_model2:
    y_pred = pm.sample_posterior_predictive(
        trace2.posterior
    )
    az.from_pymc3_predictions(
        y_pred, idata_orig=trace2, inplace=True
    )
with main_model2:
    ppc2 = pm.sample_posterior_predictive(
        trace2, var_names=[f'theta_genre_{genre}_{i}' for genre in genre_list for i in range(1+p_num)]+['saved_genre']
    )

In [ ]:
# posterior distribution of beta
beta_list = [f'beta_{genre}' for genre in genre_list]
az.plot_trace(trace2, var_names=beta_list, combined=True)

In [ ]:
# predictive posterior distribtuion
ax = sns.distplot(y_pred['popularity'].flatten())
ax.set(xlabel='Y|X', ylabel='Probability')
plt.show()